<a href="https://colab.research.google.com/github/TharunikaaM/Dimensionality-Reduction-using-K-Means-for-Activity-Recognition/blob/main/Dimensionality_Reduction_using_K_Means_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

print(df.head())
print(df.describe())
print(df.isnull().sum())
print(df.info())

<ipython-input-3-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-3-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


        0         1         2         3         4         5         6    \
0  0.288585 -0.020294 -0.132905 -0.995279 -0.983111 -0.913526 -0.995112   
1  0.278419 -0.016411 -0.123520 -0.998245 -0.975300 -0.960322 -0.998807   
2  0.279653 -0.019467 -0.113462 -0.995380 -0.967187 -0.978944 -0.996520   
3  0.279174 -0.026201 -0.123283 -0.996091 -0.983403 -0.990675 -0.997099   
4  0.276629 -0.016570 -0.115362 -0.998139 -0.980817 -0.990482 -0.998321   

        7         8         9    ...       551       552       553       554  \
0 -0.983185 -0.923527 -0.934724  ... -0.074323 -0.298676 -0.710304 -0.112754   
1 -0.974914 -0.957686 -0.943068  ...  0.158075 -0.595051 -0.861499  0.053477   
2 -0.963668 -0.977469 -0.938692  ...  0.414503 -0.390748 -0.760104 -0.118559   
3 -0.982750 -0.989302 -0.938692  ...  0.404573 -0.117290 -0.482845 -0.036788   
4 -0.979672 -0.990441 -0.942469  ...  0.087753 -0.351471 -0.699205  0.123320   

        555       556       557       558       559       560  
0  0

In [ ]:
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())
print(encoded_y)

[4 4 4 ... 1 1 1]


In [ ]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
X_train_full, X_test_full, y_train, y_test = train_test_split(df_scaled,y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB
start_time = time.time()
pipeline = Pipeline([
    ('classifier', GaussianNB())
])
pipeline.fit(X_train_full, y_train.values.ravel())
y_pred = pipeline.predict(X_test_full)
end_time = time.time()
total_time = end_time - start_time
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy{accuracy:.4f}")
print(f"Total time taken{total_time:.4f}")

Accuracy0.7315
Total time taken0.0568


In [39]:
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df_scaled[:, selected_features_indices]
print(selected_features)

[[ 2.73387413 -0.98538569 -0.8868461  ... -0.71840922  3.41252516
  -0.70666073]
 [ 0.59841104 -0.98538569 -0.87441215 ...  0.24525519  2.37604697
  -0.70666073]
 [ 0.59538848 -0.98538569 -0.86722584 ... -0.67367648  1.27125913
   2.39862783]
 ...
 [-1.42923149  0.94563179  1.11351658 ...  0.4560065  -0.8488946
  -0.04124171]
 [-1.87612102  0.84349125  1.09399383 ...  0.23656391 -1.16992694
  -0.04124171]
 [-2.00783596  0.83530308  1.2478795  ...  0.15363336 -1.06466879
  -0.04124171]]


In [40]:
X_train_selected, X_test_selected, y_train, y_test = train_test_split(selected_features, encoded_y, test_size=0.2, random_state=42)

In [41]:
start_time = time.time()
gnb_classifier = GaussianNB()
gnb_classifier.fit(X_train_selected,y_train)
end_time = time.time()
total_time = end_time - start_time
print(f"Total time taken{total_time:.4f}")

Total time taken0.0086


In [42]:
y_pred = gnb_classifier.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8165
